In [1]:
# -*- coding: utf-8 -*-
import jieba
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import numpy as np
import json
from collections import defaultdict,Counter
from pprint import pprint
from sklearn import preprocessing
import math
keyTypes = ["user profile","likes","all feed","attending events","not_replied events","declined events","friends","tagged photos","uploaded photos","checkin"]

ModuleNotFoundError: No module named 'jieba'

In [3]:
from gensim.models import word2vec
from gensim import models
import logging

D:\Users\Yun Hsuan\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = models.Word2Vec.load('word2vec-tutorial-master\\word2vec.model')

2018-05-24 16:24:54,591 : INFO : loading Word2Vec object from word2vec-tutorial-master\word2vec.model
2018-05-24 16:25:02,382 : INFO : loading trainables recursively from word2vec-tutorial-master\word2vec.model.trainables.* with mmap=None
2018-05-24 16:25:02,384 : INFO : loading syn1neg from word2vec-tutorial-master\word2vec.model.trainables.syn1neg.npy with mmap=None
2018-05-24 16:25:06,414 : INFO : loading wv recursively from word2vec-tutorial-master\word2vec.model.wv.* with mmap=None
2018-05-24 16:25:06,416 : INFO : loading vectors from word2vec-tutorial-master\word2vec.model.wv.vectors.npy with mmap=None
2018-05-24 16:25:12,073 : INFO : setting ignored attribute vectors_norm to None
2018-05-24 16:25:12,078 : INFO : loading vocabulary recursively from word2vec-tutorial-master\word2vec.model.vocabulary.* with mmap=None
2018-05-24 16:25:12,079 : INFO : setting ignored attribute cum_table to None
2018-05-24 16:25:12,082 : INFO : loaded word2vec-tutorial-master\word2vec.model


In [5]:
li = []
dataSet = defaultdict(lambda:defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: dict()))))
# f為所有人的編號、認證碼、fb ID的對照表
f = open('idlist.csv', 'r')
for row in csv.reader(f):
    fileName = 'Data\\%s.txt' % row[0]
    row[2] = row[2].lower()
    row[2] = row[2].replace(" ", "")
    print(row[2], row[0])
    data = open(fileName, 'r', encoding='UTF-8')
    myDict = defaultdict(lambda: defaultdict(lambda: dict()))
    for line in data:
        for keyType in keyTypes:
            if keyType in line:
                print(keyType)
                #idx = 0
                while True:
                    x = data.readline()
                    if '{' not in x:
                        break
                    d = x.replace('\n','')
                    d = d.replace(r"\’", r"\\'")
                    text = json.loads(d.replace('\u3000', ''))
                    myId = text['id']
                    #print(myId)
                    myDict[keyType][myId] = text                   
                break
    key = row[2]
    dataSet[key][0] = row[0]
    dataSet[key][1] = myDict
    data.close()
f.close()

lingo 1648558451902853
user profile
likes
all feed
lingo 1648558451902853
user profile
likes
all feed
lingo 1648558451902853
user profile
likes
all feed
lingo 1648558451902853
user profile
likes
all feed
lingo 1648558451902853
user profile
likes
all feed
lingo 1648558451902853
user profile
likes
all feed
123 1648558451902853
user profile
likes
all feed
lingo 1648558451902853
user profile
likes
all feed
123 1648558451902853
user profile
likes
all feed
56+565 1648558451902853
user profile
likes
all feed
s1 1648558451902853
user profile
likes
all feed
123 1648558451902853
user profile
likes
all feed
104062101 1467138566673036
user profile
likes
all feed
attending events
not_replied events
declined events
friends
tagged photos
uploaded photos
checkin
p10010 1870170133006707
user profile
likes
all feed
p10002 897809303734055
user profile
likes
all feed
attending events
not_replied events
declined events
friends
tagged photos
uploaded photos
checkin
p10002 897809303734055
user profile
likes


attending events
not_replied events
declined events
friends
tagged photos
uploaded photos
checkin
p20036 1253749398091728
user profile
likes
all feed
p20086 2100894889938310
user profile
likes
all feed
p20043 937303066443594
user profile
likes
all feed
attending events
not_replied events
declined events
friends
tagged photos
uploaded photos
checkin
p20099 2122485877766618
user profile
likes
all feed
p20030 10216636096681369
user profile
likes
all feed
p20011 10211172137530496
user profile
likes
all feed
p20041 941015479391906
user profile
likes
all feed
p20031 1685522951492331
user profile
likes
all feed
p20040 1826599634025900
user profile
likes
all feed
p20049 1766038510087307
user profile
likes
all feed
p20017 1577130962307535
user profile
likes
all feed
attending events
not_replied events
declined events
friends
tagged photos
uploaded photos
checkin
p20012 1893476847348378
user profile
likes
all feed
p20046 1980384742003363
user profile
likes
all feed
p20051 1856809014331490
user p

In [6]:
flag = 0
# f1為要標label的人的編號 ex. pxxxx
f1 = open('Data/idlist422.csv', 'r')
# jieba custom setting.
jieba.set_dictionary('dict.txt.big')
tf_idf_Dict = defaultdict(lambda: list())
### location
for row in csv.reader(f1):
    iid = row[0]
    x = dataSet[iid][0]
    num = 0
    if 'all feed' in dataSet[iid][1]:
        allfeed = dataSet[iid][1]['all feed']
        corpus = []
        for feedID in allfeed:
            if 'message' in allfeed[feedID]:
                line = allfeed[feedID]['message']
                corpus.append(" ".join(jieba.cut(line.split(',')[0], cut_all=False)))
        vectorizer = TfidfVectorizer()
        print(corpus)
        try:
            tfidf = vectorizer.fit_transform(corpus)
            print(tfidf.shape)

            words = vectorizer.get_feature_names()
            for i in range(len(corpus)):
                #print('----Document %d----' % (i))
                for j in range(len(words)):
                    if tfidf[i,j] > 10e-5:
                        tf_idf_Dict[iid].append(words[j])
            print(iid)
        except:
            print(iid,'x')
    else:
        print(iid,'x')

Building prefix dict from D:\Users\Yun Hsuan\Google Drive\Documents\School\School Project\Feature testing\dict.txt.big ...
2018-05-24 16:25:48,329 : DEBUG : Building prefix dict from D:\Users\Yun Hsuan\Google Drive\Documents\School\School Project\Feature testing\dict.txt.big ...
Loading model from cache C:\Users\YUNHSU~1\AppData\Local\Temp\jieba.ufe7737252ad35fd6e0270e13a0de67bd.cache
2018-05-24 16:25:48,366 : DEBUG : Loading model from cache C:\Users\YUNHSU~1\AppData\Local\Temp\jieba.ufe7737252ad35fd6e0270e13a0de67bd.cache
Loading model cost 3.262 seconds.
2018-05-24 16:25:51,594 : DEBUG : Loading model cost 3.262 seconds.
Prefix dict has been built succesfully.
2018-05-24 16:25:51,596 : DEBUG : Prefix dict has been built succesfully.


(252, 1860)
p10002
(63, 823)
p10008
(198, 2573)
p10010
(14, 172)
p10022
(6, 24)
p10027
p10034 x
(244, 1048)
p10035
(197, 1209)
p10036
(33, 828)
p10043
(13, 419)
p10051
(742, 6621)
p10056
(71, 956)
p10058
(46, 511)
p10059
(605, 2788)
p10062
(66, 1357)
p10069
(20, 133)
p10077
(182, 2145)
p10082
(1414, 6019)
p10046
(126, 720)
p10003
p10064 x
(633, 1122)
p10005
(73, 909)
p10018
p10019 x
(880, 4016)
p10028
(61, 745)
p10037
(296, 1739)
p10039
(187, 1624)
p10045
p10071 x
(330, 1214)
p10075
(408, 3061)
p10076
(16, 313)
p10078
(259, 1217)
p10029
(116, 1213)
p10052
(104, 806)
p10072
(196, 1393)
p10041
(104, 1120)
p10050
(121, 1194)
p10066
(409, 3892)
p10013
(13, 83)
p10014
(394, 3178)
p10015
p10042 x
(750, 5072)
p20003
(320, 1940)
p20007
(32, 877)
p20010
(212, 1327)
p20011
p20013 x
(323, 2125)
p20018
(414, 1902)
p20021
(166, 676)
p20025
(55, 1385)
p20027
p20028 x
(636, 2058)
p20030
p20037 x
(40, 449)
p20040
(287, 2096)
p20059
(1, 17)
p20065
p20077 x
(227, 2819)
p20024
(1582, 14000)
p20031
(97, 3

In [10]:
import json
with open('result.json', 'w',encoding='utf-8') as fp:
    json.dump(tf_idf_Dict, fp)

In [11]:
for i in open('result.json', 'r',encoding='utf-8'):
    print(json.loads(i))

{'p10002': ['七月', '因為', '女孩', '比較', 'lucky', '不過', '任務', '方式', '這個', '達成', '頭好', '高鐵', 'xd83d', 'xde03', '一個', '一大', '一如', '一早', '三個', '三八婦女節', '三月', '不該', '人權', '什麼', '代表', '何嘗', '充斥', '其實', '別忘了', '到底', '削減', '創造', '可以', '同性戀者', '同理', '基本', '女性', '女權', '希望', '平等', '弱勢', '情境', '想要', '意義', '我們', '所謂', '打折', '摘錄', '擁有', '文章', '時間', '更好', '有所', '期待', '機會', '權利', '權力', '每回', '沒有', '滿滿', '爭取', '特賣', '生活', '男性', '男權', '異性戀', '相愛', '社會', '網路', '職位', '職場', '臉書', '自主', '自由', '蛋糕', '行銷', '製造', '覺得', '角色', '許個', '變成', '賦予', '越大越', '逐漸', '還有', '醒來', '重大', '願好', '願望', '鼓勵', '一番', '一部', '七月', '上映', '再來', '品味', '喜歡', '安生', '心底', '快要', '情感', '戲院', '無限', '看到', '覺得', '赤裸', '長片', '雞皮疙瘩', '震撼', '壓歲錢', '葉逸凡', '角落', '快來', '抽扭蛋', '泰勇', '2018', '台北', '晚會', '跨年', '2018', '台北', '晚會', '跨年', '3d', 'professional3dprints', '列印', '初音', '專業', '工坊', '2018', '台北', '晚會', '跨年', '2018', '台北', '晚會', '跨年', '2018', '台北', '晚會', '跨年', '2018', '台北', '晚會', '跨年', '2018', '台北', '晚會', '跨年', '保護', '好好', '真的', '自己', '出去', '自己', 'xd8

In [25]:
keyList = []
keys = ['食物', '健康', '愛吃', '暴食', '宵夜','身材']
keyList+=[ [x[0],x[1]] for x in zip(*[keys[i:] for i in range(2) if i <= len(keys) ] ) ]
for k in keys:
    keyList.append([k])
print(keyList)

[['食物', '健康'], ['健康', '愛吃'], ['愛吃', '暴食'], ['暴食', '宵夜'], ['宵夜', '身材'], ['食物'], ['健康'], ['愛吃'], ['暴食'], ['宵夜'], ['身材']]


In [16]:
foodList_Dict = defaultdict(lambda: list())

In [17]:
for key in keyList: 
    for topnValue in [100,200,300,400,500]:
        print(key)
        res = model.most_similar(positive=key,topn = topnValue)
        foodList = [item[0] for item in res]
        print(foodList)

['食物', '健康']


D:\Users\Yun Hsuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


['營養', '膳食', '素食', '食品', '身體健康', '餐桌上', '保健', '飲食', '飲水', '肉類', '有益', '飲食習慣', '食材', '母乳', '適量', '有害', '脂肪', '餵食', '营养', '家禽', '飲用水', '三餐', '攝取', '進食', '食用', '飲用', '嬰幼兒', '二手菸', '衣物', '吸菸', '衛生', '消化', '心理健康', '餐點', '植物性', '烹煮', '主食', '素食者', '咖啡豆', '卡路里', '戒菸', '補充劑', '正餐', '飼料', '乳製品', '改善生活', '海鮮類', '補充品', '營養素', '身心健康', '用藥', '补充品', '吃', '家畜', '防病', '糞便', '營養學', '果汁', '血液', '伙食', '牛奶', '高蛋白', '煮食', '含酒精', '食慾', '酒精', '飽足', '吃喝', '甜食', '蔬菜水果', '戒煙', '水果', '肉食', '烹調', '生活必需品', '高熱量', '飯菜', '消費者', '睡眠', '保護環境', '不吃', '早餐', '不耐症', '飲品', '乾淨', '蔬菜', '排泄物', '豬隻', '慢性病', '餐', '飼養者', '火雞肉', '必需品', '咀嚼', '排便', '益生菌', '哺乳', '減肥', '烹飪', '殺菌']
['食物', '健康']
['營養', '膳食', '素食', '食品', '身體健康', '餐桌上', '保健', '飲食', '飲水', '肉類', '有益', '飲食習慣', '食材', '母乳', '適量', '有害', '脂肪', '餵食', '营养', '家禽', '飲用水', '三餐', '攝取', '進食', '食用', '飲用', '嬰幼兒', '二手菸', '衣物', '吸菸', '衛生', '消化', '心理健康', '餐點', '植物性', '烹煮', '主食', '素食者', '咖啡豆', '卡路里', '戒菸', '補充劑', '正餐', '飼料', '乳製品', '改善生活', '海鮮類', '補充品', '營養素', '身心健康', '用藥', '补充品', '吃', '家畜

['好吃', '營養', '甜食', '做菜', '優格', '紅蘿蔔', '靠得住', '零食', '味覺', '冰淇淋', '減肥', '身體健康', '三餐', '挑食', '荷包蛋', '果汁', '胡蘿蔔', '原味', '蝦子', '抽菸', '不吃', '餐', '煮飯', '飲食習慣', '果凍', '痘痘', '做飯', '過敏', '口香糖', '甜品', '布丁', '烹飪', '大餐', '喝牛奶', '蛋包飯', '可口', '嗜好', '貪吃', '零嘴', '青椒', '飯後', '外食', '素食', '餅乾', '蓋飯', '好食', '紅茶', '海鮮類', '青瓜', '牛奶', '胃口', '蜜糖', '飯菜', '抽煙', '家常', '飲食', '餐桌上', '保健', '嬰幼兒', '飲品', '清淡', '醃', '奶', '飯盒', '雞肉', '看電視', '漢堡包', '吃', '嚼', '肥胖', '新鮮', '小菜', '牛乳', '水果', '吃錯', '米飯', '滋補', '乳製品', '養身', '偷吃', '育兒', '肉類', '染髮', '醬', '薯片', '很會', '辣味', '金槍魚', '吃零食', '美白', '戒菸', '燕麥', '菜色', '奶油', '辛辣', '頭腦好', '甜點', '糖果', '開胃', '荻田寛子', '餵食', '花生', '黑輪', '湯品', '飲料', '奶昔', '招牌菜', '紅酒', '冰棒', '低脂', '生魚片', '白飯', '戒煙', '飯糰', '食物', '笑口常開', '衛生棉', '節食', '對貓', '竹輪', '小朋友', '咖哩', '煙燻', '肉包', '舒服', '泡澡', '湯圓', '小雞', '海苔', '蔬果', '牛排', '難吃', '秘訣', '桂格', '玩樂', '關東煮', '蛋黃', '財運', '討厭', '年糕', '薯條', '尿布', '肌膚', '喝咖啡', '海產', '香腸', '小孩子', '素菜', '馬鈴薯', '速食', '火雞肉', '下午茶', '西瓜', '變胖', '軟糖', '很壞', '飼主', '咖啡豆', '素食者', '雞蛋', '幫人', '鮮

['貪吃', '懶惰', '甜食', '調皮', '噁心', '味覺', '沒輒', '討厭', '對貓', '愛哭', '怪癖', '使喚', '青椒', '胡蘿蔔', '捉弄', '紅蘿蔔', '膽小', '纏著', '愛哭鬼', '為伴', '難吃', '偷吃', '易怒', '渾身', '變態', '自我中心', '蟲子', '肉球', '貧乳', '羨慕', '小鈴', '遲鈍', '很怕', '不吃', '變胖', '淫蕩', '挑食', '狸貓', '肚子餓', '潔癖', '流口水', '食量', '平胸', '禿頭', '兇暴', '為樂', '蜜糖', '小雞', '蘿莉控', '病態', '舔', '打噴嚏', '壞心眼', '吃醋', '癖好', '兄控', '做菜', '咬著', '滿臉', '如命', '誤食', '貓咪', '貪食', '小尊', '好勝', '蟑螂', '毛毛蟲', '很胖', '對人', '天真無邪', '神經質', '貓妖', '性情溫和', '自戀狂', '戲弄', '帶點', '吃零食', '抓傷', '阿籬', '過敏', '貪睡', '莓', '虐狂', '嗜好', '虛榮', '鬥嘴', '粗魯', '嚇到', '食慾', '忌妒', '力大無窮', '辛辣', '蝦子', '圓滾滾', '迷迷糊糊', '嘴裡', '毒舌', '懼高症', '過動', '家貓']
['愛吃', '暴食']
['貪吃', '懶惰', '甜食', '調皮', '噁心', '味覺', '沒輒', '討厭', '對貓', '愛哭', '怪癖', '使喚', '青椒', '胡蘿蔔', '捉弄', '紅蘿蔔', '膽小', '纏著', '愛哭鬼', '為伴', '難吃', '偷吃', '易怒', '渾身', '變態', '自我中心', '蟲子', '肉球', '貧乳', '羨慕', '小鈴', '遲鈍', '很怕', '不吃', '變胖', '淫蕩', '挑食', '狸貓', '肚子餓', '潔癖', '流口水', '食量', '平胸', '禿頭', '兇暴', '為樂', '蜜糖', '小雞', '蘿莉控', '病態', '舔', '打噴嚏', '壞心眼', '吃醋', '癖好', '兄控', '做菜', '咬著', '滿臉', 

...

['食材', '肉類', '餐桌上', '主食', '食用', '烹煮', '營養', '進食', '正餐', '甜食', '飯菜', '吃', '膳食', '餐點', '飲水', '排泄物', '腐肉', '糞便', '適量', '魚餌', '餵食', '咖啡豆', '家禽', '烹調', '生食', '配菜', '衣物', '乳製品', '蔬菜', '火雞肉', '魚肉', '飼料', '主餐', '水果', '家畜', '磨碎', '不吃', '物品', '咀嚼', '調味料', '養分', '菜餚', '海鮮類', '牛油', '橄欖油', '生肉', '飲用', '植物性', '柴火', '肉食', '鹿肉', '餡餅', '高熱量', '蔬菜水果', '飲用水', '米飯', '野果', '三餐', '可食用', '醃製', '雞肉', '餵', '吃剩', '養份', '禽肉', '母乳', '食法', '甜味', '配料', '牲畜', '肉', '飲食習慣', '喝水', '享用', '煮熟', '肉汁', '生吃', '宰殺', '卡路里', '鮮味', '佐料', '家禽類', '花蜜', '素食', '吃光', '牛肉', '穀物', '魚子醬', '攝取', '酒水', '生魚片', '豆類', '熱食', '脂肪', '菜色', '豬隻', '煮食', '馬鈴薯', '油炸', '水份']
['食物']
['食材', '肉類', '餐桌上', '主食', '食用', '烹煮', '營養', '進食', '正餐', '甜食', '飯菜', '吃', '膳食', '餐點', '飲水', '排泄物', '腐肉', '糞便', '適量', '魚餌', '餵食', '咖啡豆', '家禽', '烹調', '生食', '配菜', '衣物', '乳製品', '蔬菜', '火雞肉', '魚肉', '飼料', '主餐', '水果', '家畜', '磨碎', '不吃', '物品', '咀嚼', '調味料', '養分', '菜餚', '海鮮類', '牛油', '橄欖油', '生肉', '飲用', '植物性', '柴火', '肉食', '鹿肉', '餡餅', '高熱量', '蔬菜水果', '飲用水', '米飯', '野果', '三餐', '可食用', '醃製', 

['保健', '心理健康', '身體健康', '母嬰', 'health', '營養', '身心健康', '照護', '嬰幼兒', '衛生', '戒菸', '安全', '食品安全', '低碳', '身心', '改善生活', '美容', '健檢', '愛滋', '慢性病', '活力', '戒煙', '福利', '衛教', '公共衛生', '有益', '福祉', '健全', '状况不佳', '預防', '育兒', '社會福利', '醫療保健', '保護環境', '疾病', '生活品質', '減廢', '素食', '老年人', '節能', '食品', '祝君', '不佳為', 'healthy', '體適', '糖尿病', '吳沂家', '有害', '奶粉', '足脊', '飲食', '增進法', '医疗保健', '防治', '心身', '醫療保險', '交通安全', '防癌', '公益', '失調', '捐血', '卫生', '抗癌', '保養', '醫療', '營養學', '急救', '藥物濫用', '重症', '減碳', '病童', '每况愈下', '孕前', '減肥', '節約能源', '二手菸', '環保', '愛滋病', '適能', '膳食', '兒童教育', '失能', '環境衛生', '生理', '用藥', '環境保護', '戒毒', '性病', '健美', '心血管', '就業', '心理衛生', '健康狀況', '艾滋病', '健身', '高危', '銀髮族', '病患', '保險制度', '食安', '患者', '衛生署', '人身安全', '健康檢查', '護理', '血糖', '乳癌', '失依', '更年期', '關愛', '教育', '轉差', '喬山', '消防安全', '健保', '星級', '公共利益', '福利部', '健身运动', '保健食品', '復健', '精神', '助人', '自閉症', '吸菸', '營養不良', '療護', '病人', '心六倫', '防病', '產後', '優質', '養生', '篩檢', '睡眠', '輸血', '高風險', '公益活動', '食得', '全民', '每朝', '家庭教育', '消費者', '癌症', '免疫力', '兒少', '愛心', '安全性', '宜居', '社會保險', '育

['甜食', '紅蘿蔔', '青椒', '好吃', '蛋包飯', '竹輪', '做菜', '胡蘿蔔', '肉包', '饅頭', '章魚燒', '醬', '貪吃', '蓋飯', '蘿蔔', '米飯', '關東煮', '雞肉', '油條', '偷吃', '餡', '荷包蛋', '零嘴', '討厭', '布丁', '蜜糖', '蔥', '青瓜', '黑輪', '辣味', '茄子', '醃', '高麗菜', '生魚片', '零食', '蛋黃醬', '飯糰', '芋頭', '很會', '餅乾', '咖哩', '小雞', '蝦子', '甜點', '挑食', '叫成', '難吃', '蘆筍', '白飯', '酥餅', '調皮', '很愛吃', '點心', '魷魚', '年糕', '洋蔥', '漢堡包', '餃子', '不吃', '果凍', '煮飯', '小菜', '奶油', '起司', '海苔', '起士', '甜品', '冰棒', '壽喜燒', '親手做', '餅', '吐司', '蝦仁', '香菇', '拿手菜', '柳橙', '蕎麥', '圓滾滾', '飯菜', '最怕', '靠得住', '糰子', '優格', '喜歡', '豬排', '金槍魚', '燒', '小尊', '可麗餅', '涼拌', '燉煮', '嗜好', '薯條', '紅茶', '白蘿蔔', '吃過', '黃瓜', '醬汁', '肉丸', '可口', '小黃瓜', '菠菜', '茶泡飯', '雞湯', '吃零食', '冰淇淋', '炒蛋', '湯圓', '胃王', '鬆餅', '韓式', '芹菜', '西紅柿', '超辣', '葡萄乾', '常吃', '大餐', '一碗', '濃湯', '捉弄', '鮮蝦', '品嘗', '咖喱飯', '萵苣', '奇異果', '鹿肉', '海參', '鹹蛋', '乳酪', '奶昔', '肚子餓', '馬鈴薯', '燒賣', '燙', '麻婆豆腐', '排骨', '食量', '燉肉', '牛蒡', '花椰菜', '蘋果派', '品嚐', '湯品', '香腸', '滷蛋', '餃', '鯖魚', '薯片', '做飯', '愛喝', '辛辣', '竹筍', '肉乾', '主菜', '羊羹', '菜色', '烤', '米糕', '打麻將', '花生', '魚乾', '作菜', '煎

['司教羅伊', '怠惰', '嗜血', '色欲', '昏睡', '污穢', '貪婪', '瀕死', '易怒', '失憶症', '劇毒', '性慾', '自殘', '雙重人格', '司教', '失去理性', '痛覺', '受虐', '噬魂', '暈眩', '吞食', '發狂', '人格分裂', '邪魔', '適合者', '魔性', '病態', '飢渴', 'gluttony', '邪念', '過動', '神經性', '罪者', '無意識', '怨念', '體液', '貪食', '大罪', '虛弱', '感到恐懼', '人格障礙', '幽閉', '殘忍', '噬', '邪靈', '昏厥', '罪孽', '蠱毒', '無害', '淫慾', '幻覺', '癲癇', '靈體', '精神障礙', 'krad', '精神異常', '發瘋', '瘴氣', '歇斯底里', '憑依', '之種', '夢魔', '執念', '施虐', '思覺', '怨靈', '怪蟲', '所騙', '妄想症', '肉體上', '兇殘', '肉體', '抓傷', '流著', '自私', '情欲', '附體', '毒蟲', '附身', '憎恨', '精神分裂', '壞死', '毒藥', '焦慮', '噁心', '凶暴', '熱病', '麻痺', '魔眼', '薩麥爾', '憎惡', '黑化', '之傷', '無腦', '惡心', '治癒', '強迫症', '芥邊', '異能者', '呼吸困難']
['暴食']
['司教羅伊', '怠惰', '嗜血', '色欲', '昏睡', '污穢', '貪婪', '瀕死', '易怒', '失憶症', '劇毒', '性慾', '自殘', '雙重人格', '司教', '失去理性', '痛覺', '受虐', '噬魂', '暈眩', '吞食', '發狂', '人格分裂', '邪魔', '適合者', '魔性', '病態', '飢渴', 'gluttony', '邪念', '過動', '神經性', '罪者', '無意識', '怨念', '體液', '貪食', '大罪', '虛弱', '感到恐懼', '人格障礙', '幽閉', '殘忍', '噬', '邪靈', '昏厥', '罪孽', '蠱毒', '無害', '淫慾', '幻覺', '癲癇', '靈體', '精神障礙', 'krad'

['消夜', '小菜', '大餐', '嚐', '飯菜', '餸', '品嚐', '湯圓', '下午茶', '便當', '年夜飯', '飯後', '點心', '進補', '佳餚', '三餐', '紅酒', '一碗', '米飯', '蜜糖', '雞肉', '煮飯', '偷吃', '滷蛋', '雞湯', '零食', '蝦仁', '早茶', '炒飯', '燒肉', '叫賣', '意粉', '品嘗', '煲', '叉燒飯', '試吃', '臭豆腐', '油條', '茶葉蛋', '買菜', '喝咖啡', '飯盒', '撈', '煮', '蘿蔔', '小籠包', '凍', '吃火鍋', '吃喝玩樂', '喫', '鹹魚', '滷味', '好吃', '晚市', '漢堡包', '粽', '餡', '肉餅', '粥', '主菜', '豬肉', '火鍋', '涼拌', '招牌菜', '焗', '生魚片', '魚肉', '甜品', '沙律', '採摘', '紅蘿蔔', '自助餐', '一餐', '充飢', '紅燒', '做飯', '醃', '飯時', '飲', '嚼', '白飯', '餐點', '滋補', '街邊', '曬', '煮食', '菜色', '饅頭', '海鮮', '鮑魚', '好料', '暢飲', '雲吞', '糖水', '晚飯', '叉燒', '路邊攤', '吃過', '排骨', '麻婆豆腐', '鑊', '涼粉', '水餃', '下酒菜', '小食', '吃不完', '正餐', '特飲', '家常', '剩飯', '擺攤', '飲茶', '紙巾', '炒菜', '青瓜', '玩遊戲', '烤雞', '喝一杯', '年糕', '臘味', '雞蛋', '逛街', '烹煮', '檸檬汁', '好喝', '串燒', '熱騰騰', '皮蛋', '空心菜', '糕餅', '餐', '一杯', '煮成', '蛋包飯', '壽司', '豆皮', '醬', '茶點', '剩菜', '親手做', '開水', '燉', '三碗', '雞腿', '洋蔥', '上桌', '攤', '煮出', '搵', '珍珠奶茶', '下酒', '奶茶', '夜宵', '炒', '鍋巴', '泡茶', '原味', '魚翅', '八分鐘', '鮮奶', '鮮蝦', '鹹', '前菜', '魚蛋', '關東煮', '

In [19]:
corpus = []
allfeed = dataSet['p20028'][1]['all feed']
for feedID in allfeed:
    if 'message' in allfeed[feedID]:
        line = allfeed[feedID]['message']
        corpus.append(" ".join(jieba.cut(line.split(',')[0], cut_all=False)))
try:
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(corpus)
    print(tfidf.shape)

    words = vectorizer.get_feature_names()
    for i in range(len(corpus)):
        print('----Document %d----' % (i))
        for j in range(len(words)):
            if tfidf[i,j] > 1e-5:
                  print(words[j], tfidf[i,j])
except:
    print('x')

NameError: name 'dataSet' is not defined

In [ ]:
print(corpus)

In [ ]:
# f1為要標label的人的編號 ex. pxxxx
f1 = open('Data/idlist422.csv', 'r')
### location
for row in csv.reader(f1):
    iid = row[0]
    x = dataSet[iid][0]
    print(food_num_Dict[iid])

In [20]:
# f1為要標label的人的編號 ex. pxxxx
f1 = open('Data/idlist422.csv', 'r')
### location
for row in csv.reader(f1):
    iid = row[0]
    x = dataSet[iid][0]
    pprint(iid)
    pprint(food_Dict[iid])

NameError: name 'dataSet' is not defined